In [1]:
from dotenv import load_dotenv
load_dotenv()
import rich

In [2]:
from langfuse import Langfuse
langfuse = Langfuse()

In [3]:
langfuse.create_prompt(
    name="event-planner",
    prompt=
    "Plan an event titled {{Event_Name}}. The event will be about: {{Event_Description}}. "
    "The event will be held in {{Location}} on {{Date}}. "
    "Consider the following factors: audience, budget, venue, catering options, and entertainment. "
    "Provide a detailed plan including potential vendors and logistics.",
    config={
        "model":"gpt-4o-mini",
        "temperature": 0,
    },
    labels=["production"]
)

In [4]:
langfuse_prompt = langfuse.get_prompt(
    name="event-planner",
    label="production",
    cache_ttl_seconds=30, # default is 60
    fallback="I'm sorry, I'm not sure what you're asking for. Could you please provide more details?", # default prompt if no cache and fetch is failed
    max_retries= 2 # default is 2(s), max is 4(s), max delay is 10(s)
    )
print(langfuse_prompt.prompt)

Plan an event titled {{Event_Name}}. The event will be about: {{Event_Description}}. The event will be held in {{Location}} on {{Date}}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.


In [5]:
print(langfuse_prompt.config)
print(langfuse_prompt.version)

{'model': 'gpt-4o-mini', 'temperature': 0}
3


##### Convert to LangChain format

In [6]:
langchain_prompt_template = langfuse_prompt.get_langchain_prompt()
print(langchain_prompt_template)

Plan an event titled {Event_Name}. The event will be about: {Event_Description}. The event will be held in {Location} on {Date}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.


##### Compile

In [7]:
langfuse_prompt.compile(Event_Name="My Birthday Party", Event_Description="A fun party", Location="My house", Date="Saturday")

'Plan an event titled My Birthday Party. The event will be about: A fun party. The event will be held in My house on Saturday. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.'

### Transform into Langchain PromptTemplate

In [21]:
from langchain_core.prompts import ChatPromptTemplate

langchain_prompt = ChatPromptTemplate.from_template(
    template=langfuse_prompt.get_langchain_prompt(),
    metadata={"langfuse_prompt": langfuse_prompt}
)

In [20]:
rich.print(langchain_prompt)

ChatPromptTemplate(
    input_variables=['Date', 'Event_Description', 'Event_Name', 'Location'],
    input_types={},
    partial_variables={},
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['Date', 'Event_Description', 'Event_Name', 'Location'],
                input_types={},
                partial_variables={},
                metadata={'langfuse_prompt': <langfuse.model.TextPromptClient object at 0x1065dc1a0>},
                template='Plan an event titled {Event_Name}. The event will be about: {Event_Description}. The 
event will be held in {Location} on {Date}. Consider the following factors: audience, budget, venue, catering 
options, and entertainment. Provide a detailed plan including potential vendors and logistics.'
            ),
            additional_kwargs={}
        )
    ]
)

In [10]:
model = langfuse_prompt.config["model"]
temperature = str(langfuse_prompt.config["temperature"])
print(f"Prompt model configurations\nModel: {model}\nTemperature: {temperature}")

Prompt model configurations
Model: gpt-4o-mini
Temperature: 0


In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, temperature=temperature)

chain = langchain_prompt | llm

In [12]:
example_input = {
    "Event_Name": "Wedding",
    "Event_Description": "The wedding of Julia and Alex, a charming couple who share a love for art and nature. This special day will celebrate their journey together with a blend of traditional and contemporary elements, reflecting their unique personalities.",
    "Location": "Central Park, New York City",
    "Date": "June 5, 2024"
}

In [13]:
from langfuse.callback import CallbackHandler
langfuse_callback_handler = CallbackHandler()

In [14]:
# we pass the callback handler to the chain to trace the run in Langfuse
response = chain.invoke(input=example_input,config={"callbacks":[langfuse_callback_handler]})
 
print(response.content)

### Event Plan: Wedding of Julia and Alex

**Event Title:** Wedding of Julia and Alex

**Date:** June 5, 2024

**Location:** Central Park, New York City

---

### 1. Event Overview

**Theme:** A blend of traditional and contemporary elements, celebrating art and nature.

**Audience:** Approximately 150 guests, including family, friends, and close colleagues.

**Budget:** Estimated budget of $50,000

---

### 2. Venue

**Location:** Central Park, New York City

**Specific Area:** The Conservatory Garden or the Great Lawn (depending on availability and guest count)

**Logistics:**
- **Permits:** Apply for a special event permit through the NYC Parks Department.
- **Accessibility:** Ensure the venue is accessible for all guests, including those with mobility issues.
- **Restroom Facilities:** Arrange for portable restrooms if necessary.

---

### 3. Catering Options

**Catering Style:** Buffet with a mix of traditional and contemporary dishes, including vegetarian and vegan options.

**Po